In [ ]:
#@markdown ⬅️ 点击一键运行
#@markdown Aria2完美配置魔改版

#Aria2一键完美配置脚本一键安装
!apt install wget curl ca-certificates &> /dev/null
!wget -N git.io/aria2.sh &> /dev/null && chmod +x aria2.sh &> /dev/null
!echo 1|./aria2.sh &> /dev/null

In [ ]:
#@title 下载OnedriveUploader
!wget https://cdn.jsdelivr.net/gh/MoeClub/OneList/OneDriveUploader/amd64/linux/OneDriveUploader -P /usr/local/bin/
#赋予执行权限
!chmod +x /usr/local/bin/OneDriveUploader
#下载授权文件
!wget https://raw.githubusercontent.com/oliveryi1906/mycfg/main/auth.json -P /content/

In [ ]:
#@title 下载自动上传脚本
!rm -f /root/.aria2/oduploader.*
!wget https://raw.githubusercontent.com/oliveryi1906/mycfg/main/oduploader.sh -P /root/.aria2/
#添加执行权限
!chmod 755 /root/.aria2/oduploader.sh
#更新aria2配置
!sed -i 's?on-download-complete=/root/.aria2c/clean.sh?on-download-complete=/root/.aria2/oduploader.sh?g' /root/.aria2c/aria2.conf
!sed -i 's?#on-bt-download-complete=?on-download-complete=/root/.aria2/oduploader.sh?g' /root/.aria2c/aria2.conf

In [ ]:
!sed -i 's?#on-bt-download-complete=?on-download-complete=/root/.aria2/oduploader.sh?g' /root/.aria2c/aria2.conf

In [ ]:
#@title 自动更新tracker后重启Aria2
!echo 12|./aria2.sh &> /dev/null
!echo 6|./aria2.sh &> /dev/null

In [ ]:
#@title ngrok 穿透代理
#Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip
#Aset token
#print("Copy authtoken from https://dashboard.ngrok.com/auth")
#import getpass
#authtoken = getpass.getpass()
authtoken = "289ITmiPWtRvC4e8w9WsSF79N94_2DAqdzNPFoCQFWVBHjArg"  # @param {type:"string"}

#REGION = "JP" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]



#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 6800 &')

#Get public address
#! curl -s http://localhost:4040/api/tunnels | python3 -c \
    #"import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

#查看Aria2配置
#!echo 8|./aria2.sh


In [ ]:
#@title 获取ngrok地址 端口
import requests
import json

page=requests.get('http://localhost:4040/api/tunnels')
#print(page.text)
response_json=json.loads(page.text)
#print(response_json)
public_url=response_json["tunnels"][0]["public_url"]
print(public_url)
ngrok_ip = public_url.strip("tcp://").split(":")[0]
ngrok_port = public_url.strip("tcp://").split(":")[1]

print(ngrok_ip)
print(ngrok_port)


In [ ]:
#@title 获取AriaNG链接
import subprocess
import re

command="echo 8|./aria2.sh"
#res = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT) # 使用管道
def printCmd(cmd):
    p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    set_info = []
    while True:
        result = p.stdout.readline()  #默认获取到的是二进制内容
        if result != b'':  #获取内容不为空时
          set_info.append(result.decode('utf-8').strip('\r\n'))
          #print(result.decode('utf-8').strip('\r\n'))  #UTF-8解码
        else:
            break
    return set_info
    p.stdout.close()


setting = printCmd(command)
#获取AriaNG连接
for r in setting:
  if "AriaNg 链接" in r:
    print(r)
    url = r.replace("AriaNg 链接	:","")
#print(url)
#url链接里的ip和端口需要替换为ngrok的ip和端口

# python中正则匹配，如果不加（）,默认是一个分组，group（）中的参数，如果是0，默认是所有组，即返回整个字符串
# 如果是要访问自定义的组的元素，则需要从1开始

old_ipport = r = re.search('/ws/(.*)/jsonrpc/', url).group(1)
ngrok_ipport = ngrok_ip+"/"+ngrok_port
new_url = url.replace(old_ipport,ngrok_ipport)

print(new_url)





In [ ]:
!ls -alh /root/.aria2c/

In [ ]:
#查看aria2配置文件
!cat /root/.aria2c/aria2.conf

In [ ]:
#查看下载日志
!cat /root/.aria2c/aria2.log

In [ ]:
!chmod 755 /root/.aria2/oduploader.sh